# Grafo de Engel — Implementación y Análisis

Este notebook implementa la construcción y análisis del **grafo de Engel** $\Gamma_G$ para grupos finitos.

## Teoría

### Definiciones básicas

- **Conmutador**: $[x,y] = x^{-1} y^{-1} x y$
- **n-Engel conmutador**: $[x, _ny] = [[x, _{n-1}y], y]$ (aplicar $n$ veces el conmutador con $y$)
  - $[x, _0y] = x$
  - $[x, _1y] = [x,y]$
- **Hipercentro**: $Z_\infty(G)$ — la unión de la serie central ascendente

### Grafo de Engel $\Gamma_G$

- **Vértices**: $V = G \setminus Z_\infty(G)$
- **Aristas**: $x \to y$ si existe $n \geq 1$ tal que $[x, _ny] = 1$

**Propiedad clave**: $G$ es nilpotente $\Longleftrightarrow$ $\Gamma_G$ es vacío

## 1. Funciones básicas

In [3]:
def conmutador(x, y):
    """
    Calcula el conmutador [x,y] = x^(-1) * y^(-1) * x * y
    """
    return x^(-1) * y^(-1) * x * y


def n_engel_commutator(x, y, n):
    """
    Calcula el n-Engel conmutador [x, _n y] = [x, y, y, ..., y]
    Definición recursiva:
    - [x, _0 y] = x
    - [x, _1 y] = [x, y]
    - [x, _n y] = [[x, _(n-1) y], y]
    """
    if n == 0:
        return x
    
    # Calcular recursivamente
    result = x
    for i in range(n):
        result = conmutador(result, y)
        #print(result)
    
    return result


def hay_arista_dirigida(x, y, G, max_n=None):
    """
    Verifica si hay una arista dirigida de x a y.
    Esto ocurre si [x, _n y] = 1 para algún n ≥ 1.
    La relación NO es simétrica en general.
    """
    if max_n is None:
        max_n = G.order()
    
    identity = G.identity()
    
    # Verificar si [x, _n y] = 1 para algún n
    for n in range(1, max_n + 1):
        comm = n_engel_commutator(x, y, n)
        #print(comm)
        if comm == identity:
            #print(n)
            return True
    
    return False


def calcular_hipercentro(G, max_iter=10):
    """
    Calcula Z_∞(G) usando la serie central ascendente
    """
    Z_n = G.subgroup([G.identity()])
    
    for _ in range(max_iter):
        if Z_n == G:
            break
        
        # Encontrar elementos que centralizan módulo Z_n
        elementos_nuevos = []
        for g in G:
            esta_en_centro = True
            for h in G:
                conmutador_gh = conmutador(g, h)
                if conmutador_gh not in Z_n:
                    esta_en_centro = False
                    break
            
            if esta_en_centro:
                elementos_nuevos.append(g)
        
        Z_n_plus_1 = G.subgroup(elementos_nuevos)
        
        if Z_n_plus_1 == Z_n:
            break
            
        Z_n = Z_n_plus_1
    
    return Z_n 

## 2. Construcción del grafo de Engel

In [4]:
def grafo_engel(G, verbose=True):
    """
    Construye el grafo de Engel Γ_G según la definición:
    - V = G  Z_∞(G)
    - x ~ y si [x, _n y] = 1 para algún n
    """
    if verbose:
        print("=" * 60)
        print(f"Grupo: {G}")
        print(f"Orden: {G.order()}")
    
    # Calcular hipercentro
    Z_inf = calcular_hipercentro(G)
    
    if verbose:
        print(f"\nHipercentro Z_∞(G):")
        print(f"  Orden: {Z_inf.order()}")
        if Z_inf.order() <= 10:
            print(f"  Elementos: {list(Z_inf)}")
    
    # Vértices: G \ Z_∞(G)
    vertices = [g for g in G if g not in Z_inf]
    
    if verbose:
        print(f"\nVértices (G \\ Z_∞(G)): {len(vertices)}")
        if len(vertices) <= 10:
            print(f"  {vertices}")
    
    if len(vertices) == 0:
        if verbose:
            print("\n⚠️  El grafo es vacío (Z_∞(G) = G)")
        return None, []
    
    # Crear grafo
    Gamma = DiGraph(loops=True)
    vertex_labels = {}
    
    for v in vertices:
        label = str(v)
        vertex_labels[v] = label
        Gamma.add_vertex(label)
    
    # Calcular aristas
    if verbose:
        print("\nCalculando aristas...")
    
    aristas_count = 0
    for i, x in enumerate(vertices):
        for j, y in enumerate(vertices):
            if x != y:
                if hay_arista_dirigida(x, y, G):
                    Gamma.add_edge(vertex_labels[x], vertex_labels[y])
                    aristas_count += 1

    return Gamma, vertices

## 3. Funciones de análisis y resumen

In [ ]:
def analizar_grupo(G, nombre):
    """Analiza un grupo y retorna datos básicos"""
    Z_inf = calcular_hipercentro(G)
    Gamma, vertices = grafo_engel(G, verbose=False)
    
    return {
        'nombre': nombre,
        'orden': G.order(),
        'hipercentro': Z_inf.order(),
        'vertices': len(vertices),
        'aristas': Gamma.size() if Gamma else 0,
        'fuertemente_conexo': Gamma.is_strongly_connected() if Gamma else False,
        'nilpotente': (Z_inf == G),
        'grafo_vacio': (Gamma is None)
    }


def tabla_resumen(datos_list):
    """Imprime tabla de resultados"""
    print("\n" + "="*80)
    print("TABLA DE RESULTADOS")
    print("="*80)
    print(f"{'Grupo':<10} {'|G|':<6} {'|Z∞|':<7} {'|V|':<6} {'|E|':<8} {'F.Conexo':<11} {'Nilp':<6}")
    print("-"*80)
    
    for d in datos_list:
        if d['grafo_vacio']:
            print(f"{d['nombre']:<10} {d['orden']:<6} {d['hipercentro']:<7} {'--':<6} {'--':<8} {'--':<11} {'Sí':<6}")
        else:
            fc = "Sí" if d['fuertemente_conexo'] else "No"
            nilp = "Sí" if d['nilpotente'] else "No"
            print(f"{d['nombre']:<10} {d['orden']:<6} {d['hipercentro']:<7} {d['vertices']:<6} {d['aristas']:<8} {fc:<11} {nilp:<6}")
    
    print("="*80)


def verificar_nilpotencia(datos_list):
    """Verifica: G nilpotente ⟺ Γ_G vacío"""
    print("\n" + "="*80)
    print("VERIFICACIÓN: G nilpotente ⟺ Γ_G vacío")
    print("="*80)
    
    todos_ok = True
    for d in datos_list:
        cumple = (d['nilpotente'] == d['grafo_vacio'])
        simbolo = "✓" if cumple else "✗"
        print(f"{simbolo} {d['nombre']:<10} Nilpotente: {str(d['nilpotente']):<5}  Grafo vacío: {d['grafo_vacio']}")
        if not cumple:
            todos_ok = False
    
    print("="*80)
    if todos_ok:
        print("✓ VERIFICADO: G nilpotente ⟺ Γ_G vacío")
    else:
        print("✗ CONTRAEJEMPLO ENCONTRADO")
    print()

## 4. Funciones estadísticas (flujo y ciclos)

In [ ]:
def flujo_maximo(G):
    if G is None or G.order() == 0:
        return 0
    out_deg = [G.out_degree(v) for v in G.vertices()]
    return max(out_deg)

def flujo_minimo(G):
    if G is None or G.order() == 0:
        return 0
    out_deg = [G.out_degree(v) for v in G.vertices()]
    return min(out_deg)

def flujo_promedio(G):
    if G is None or G.order() == 0:
        return 0
    out_deg = [G.out_degree(v) for v in G.vertices()]
    return sum(out_deg) / len(out_deg)

def tiene_ciclos(G):
    if G is None:
        return False
    return not G.is_directed_acyclic()

def contar_ciclos(G):
    """Cuenta el número de ciclos simples (puede ser costoso para grafos grandes)"""
    if G is None:
        return 0
    try:
        ciclos = G.all_simple_cycles()
        return len(ciclos)
    except:
        return -1  # Error o muy costoso

def tabla_estadisticas_flujo(grupos_lista):
    print("\n" + "="*80)
    print("ESTADÍSTICAS DE FLUJO DEL GRAFO DE ENGEL")
    print("="*80)
    print(f"{'Grupo':<10} {'F.Máx':<10} {'F.Mín':<10} {'F.Prom':<12} {'Ciclos':<10}")
    print("-"*80)
    
    for nombre, G in grupos_lista:
        Gamma, vertices = grafo_engel(G, verbose=False)
        
        if Gamma is None:
            print(f"{nombre:<10} {'--':<10} {'--':<10} {'--':<12} {'--':<10}")
        else:
            f_max = flujo_maximo(Gamma)
            f_min = flujo_minimo(Gamma)
            f_prom = flujo_promedio(Gamma)
            ciclos = "Sí" if tiene_ciclos(Gamma) else "No"
            
            print(f"{nombre:<10} {f_max:<10} {f_min:<10} {f_prom:<12.2f} {ciclos:<10}")
    
    print("="*80)

---

# APLICACIONES Y EJEMPLOS

## 5. Análisis masivo de grupos

In [ ]:
# Definir grupos a estudiar
grupos = [
    # Simétricos
    ("S_3", SymmetricGroup(3)),
    ("S_4", SymmetricGroup(4)),
    ("S_5", SymmetricGroup(5)),
    # Alternantes
    ("A_3", AlternatingGroup(3)),
    ("A_4", AlternatingGroup(4)),
    # Diédricos
    ("D_3", DihedralGroup(3)),
    ("D_4", DihedralGroup(4)),
    ("D_5", DihedralGroup(5)),
    ("D_6", DihedralGroup(6)),
    ("D_7", DihedralGroup(7)),
    ("D_8", DihedralGroup(8)),
    # Nilpotentes (para verificar la propiedad)
    ("C_4", CyclicPermutationGroup(4)),
    ("C_6", CyclicPermutationGroup(6)),
    ("Q_8", QuaternionGroup()),
]

# Analizar todos
print("ANALIZANDO GRUPOS...\n")
resultados = []
for nombre, G in grupos:
    print(f"  {nombre}...", end=" ")
    datos = analizar_grupo(G, nombre)
    resultados.append(datos)
    print("✓")

# Mostrar resultados
tabla_resumen(resultados)
verificar_nilpotencia(resultados)

print("\nOBSERVACIONES:")
print("- Para S_n: ¿|V| = |G| - 1?")
print("- Para D_n: ¿Diferencia entre n par e impar?")
print("- ¿Todos los nilpotentes tienen grafo vacío?")

## 6. Análisis estadístico: grupos diédricos

In [ ]:
# Grupos diédricos D_2 a D_16
grupos_diedricos = [(f"D_{n}", DihedralGroup(n)) for n in range(2, 17)]

# Tabla de flujo
tabla_estadisticas_flujo(grupos_diedricos)

# Análisis completo
print("\nANALIZANDO GRUPOS DIÉDRICOS...\n")
resultados_diedricos = []
for nombre, G in grupos_diedricos:
    print(f"  {nombre}...", end=" ")
    datos = analizar_grupo(G, nombre)
    resultados_diedricos.append(datos)
    print("✓")

tabla_resumen(resultados_diedricos)
verificar_nilpotencia(resultados_diedricos)

### Conteo de ciclos para grupos diédricos

In [ ]:
print("Conteo de ciclos en grafos de Engel de grupos diédricos:\n")
for nombre, grupo in grupos_diedricos[1:9]:  # D_2 a D_9
    Gamma, vertices = grafo_engel(grupo, verbose=False)
    num_ciclos = contar_ciclos(Gamma)
    print(f"{nombre}: {num_ciclos} ciclos")

## 7. Análisis estadístico: grupos simétricos

In [ ]:
grupos_simetricos = [
    ("S_3", SymmetricGroup(3)),
    ("S_4", SymmetricGroup(4)),
    ("S_5", SymmetricGroup(5)),
]

# Tabla de flujo
tabla_estadisticas_flujo(grupos_simetricos)

# Conteo de ciclos
print("\nConteo de ciclos en grafos de Engel de grupos simétricos:\n")
for nombre, grupo in grupos_simetricos:
    Gamma, vertices = grafo_engel(grupo, verbose=False)
    num_ciclos = contar_ciclos(Gamma)
    print(f"{nombre}: {num_ciclos} ciclos")

## 8. Visualizaciones individuales

Visualización del grafo de Engel para un grupo específico.

In [ ]:
# Seleccionar grupo a visualizar
G = DihedralGroup(12)  # Cambiar por: SymmetricGroup(4), AlternatingGroup(4), etc.

Gamma_G, vertices_G = grafo_engel(G)

if Gamma_G is not None:
    print("\nAnálisis de simetría:")
    aristas = list(Gamma_G.edges(labels=False)) 
    for u, v in aristas[:5]:
        tiene_reciproca = Gamma_G.has_edge(v, u)
        print(f"  {u} → {v}: recíproca {v} → {u} = {tiene_reciproca}")

    Gamma_G.show(figsize=8, vertex_size=1000, title=f"Grafo de Engel de {G}")